In [ ]:
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from multiprocessing.dummy import Pool

In [ ]:
url = 'https://www.notebooksbilliger.de/notebooks/page/{}?perPage=50&availability=sofort'

In [ ]:
n = 7
queries = list(range(1, 32))
chunks = [queries[x:x+n] for x in range(0, len(queries), n)]
len(chunks)

In [ ]:
def search(queries):
    links = []
    driver = webdriver.Firefox()
    time.sleep(8)
    for i in queries:
        driver.get(url.format(i))
        time.sleep(4)
        html = driver.page_source
        try:
            bs = BeautifulSoup(html, 'lxml').find('div', {'id':'listing'})
            for elem in bs.findAll('a', {'class':'left'}):
                links.append(elem['href'])
        except:
            pass
    driver.close()
    return links

In [ ]:
pool = Pool(5)
futures = []
for query in chunks:
    futures.append(pool.apply_async(search, [query]))
    
data = [link.get() for link in futures]
links = [item for sublist in data for item in sublist]
print('Links: {}'.format(len(links)))

In [ ]:
n = 292
chunks = [links[x:x+n] for x in range(0, len(links), n)]
print('Chunks:', len(chunks))

for i in chunks:
    print(len(i))

In [ ]:
def load(query):
    driver = webdriver.Firefox()
    time.sleep(8)
    data = []
    for link in query:
    
        try:
            driver.get(link)
            time.sleep(4)
            html = driver.page_source
            bs = BeautifulSoup(html, 'lxml')
    
            product = {}
            product['Link'] = link
            product['Preis'] = bs.find('div', {'id':'product_detail_price'}).find('text', {'class':'nbb-svg-base'}).get_text()
            for row in bs.find('table', {'class':'properties_table'}).findAll('tr'):
                try:
                    product[row.find('td', {'class':'produktDetails_eigenschaft2'}).get_text().strip()] = row.find('td', {'class':'produktDetails_eigenschaft3'}).get_text().strip()
                except:
                    pass
            data.append(product)
        except:
            pass
    driver.close()
    return data

In [ ]:
pool = Pool(5)
futures = []
for query in chunks:
    futures.append(pool.apply_async(load, [query]))
    
data = [link.get() for link in futures]
products = [item for sublist in data for item in sublist]
print('Links: {}'.format(len(products)))

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(products)
print(df.shape)
df.head()

In [ ]:
df.columns

In [ ]:
work = df[['Grafikkarte', 'Größe', 'Kapazität (Gesamt)', 'Preis', 'max. Erweiterung auf', 'Link']].dropna()
work['Größe'] = work['Größe'].apply(lambda x: x.split(' ')[0]).astype(int)
work['Kapazität (Gesamt)'] = work['Kapazität (Gesamt)'].apply(lambda x: x.split(' ')[0]).astype(int)
work['max. Erweiterung auf'] = work['max. Erweiterung auf'].apply(lambda x: x.split(' ')[0])
work = work[work['max. Erweiterung auf']!='keine']
work['max. Erweiterung auf'] = work['max. Erweiterung auf'].astype(int)
work['Preis'] = work['Preis'].apply(lambda x: x.replace('.', '').replace(',', '.')).astype(float)
work = work[work['Grafikkarte'].apply(lambda x: True if 'GeForce' in x else False)].groupby('Grafikkarte').agg({'Größe':'mean', 'Kapazität (Gesamt)':'mean', 'Preis':'mean', 'max. Erweiterung auf':'mean', 'Link':'count'}).sort_values('Preis')
work

In [ ]:
df.to_csv('Laptops.csv', index=False)